# **Cluster analysis**

### Get data from github:

In [1]:
!wget https://raw.githubusercontent.com/shitkov/cluster_analysis/master/ria_1k.csv

--2021-11-23 10:03:59--  https://raw.githubusercontent.com/shitkov/cluster_analysis/master/ria_1k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3723692 (3.6M) [text/plain]
Saving to: ‘ria_1k.csv’

ria_1k.csv          100%[===================>]   3.55M  --.-KB/s    in 0.05s   

2021-11-23 10:03:59 (68.9 MB/s) - ‘ria_1k.csv’ saved [3723692/3723692]



### Explore dataset

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('/content/ria_1k.csv')

In [4]:
data.columns

Index(['text', 'headline'], dtype='object')

In [5]:
sentences = list(set(list(data['headline'])))

### Get embeddings

In [6]:
!pip install transformers sentencepiece

     |████████████████████████████████| 3.1 MB 7.2 MB/s 
     |████████████████████████████████| 1.2 MB 18.2 MB/s 
     |████████████████████████████████| 895 kB 36.5 MB/s 
     |████████████████████████████████| 59 kB 6.8 MB/s 
     |████████████████████████████████| 596 kB 17.5 MB/s 
     |████████████████████████████████| 3.3 MB 45.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [7]:
import torch

In [8]:
from transformers import AutoTokenizer, AutoModel

In [9]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/806 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/509k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [10]:
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

Downloading:   0%|          | 0.00/492M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
import numpy as np

In [12]:
embeddings_list = []

for s in sentences:
    encoded_input = tokenizer(s, padding=True, truncation=True, max_length=64, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    embedding = model_output.pooler_output
    embeddings_list.append((embedding)[0].numpy())

embeddings = np.asarray(embeddings_list)

### Get optimum number of clusters

In [13]:
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity

def determine_k(embeddings):
    k_min = 10
    
    clusters = [x for x in range(2, k_min * 11)]
   
    metrics = []
    
    for i in clusters:
        metrics.append((KMeans(n_clusters=i).fit(embeddings)).inertia_)
    
    k = elbow(k_min, clusters, metrics)
    return k

def elbow(k_min, clusters, metrics):
    score = []

    for i in range(k_min, clusters[-3]):
        y1 = np.array(metrics)[:i + 1]
        y2 = np.array(metrics)[i:]
    
        df1 = pd.DataFrame({'x': clusters[:i + 1], 'y': y1})
        df2 = pd.DataFrame({'x': clusters[i:], 'y': y2})
    
        reg1 = LinearRegression().fit(np.asarray(df1.x).reshape(-1, 1), df1.y)
        reg2 = LinearRegression().fit(np.asarray(df2.x).reshape(-1, 1), df2.y)

        y1_pred = reg1.predict(np.asarray(df1.x).reshape(-1, 1))
        y2_pred = reg2.predict(np.asarray(df2.x).reshape(-1, 1))    
        
        score.append(mean_squared_error(y1, y1_pred) + mean_squared_error(y2, y2_pred))

    return np.argmin(score) + k_min

In [14]:
k_opt = determine_k(embeddings)

In [15]:
kmeans = KMeans(n_clusters = k_opt, random_state = 42).fit(embeddings)
kmeans_labels = kmeans.labels_

In [16]:
len(embeddings)

995

In [17]:
data = pd.DataFrame()
data['text'] = sentences
data['label'] = kmeans_labels
data['embedding'] = list(embeddings)

In [18]:
# Get nearest sentences to each center sentenses
from sklearn.metrics.pairwise import euclidean_distances

kmeans_centers = kmeans.cluster_centers_
top_texts_list = []
for i in range (0, k_opt):
    cluster = data[data['label'] == i]
    embeddings = list(cluster['embedding'])
    texts = list(cluster['text'])
    distances = [euclidean_distances(kmeans_centers[0].reshape(1, -1), e.reshape(1, -1))[0][0] for e in embeddings]
    scores = list(zip(texts, distances))
    top_3 = sorted(scores, key=lambda x: x[1])[:3]
    top_texts = list(zip(*top_3))[0]
    top_texts_list.append(top_texts)

In [19]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
MODEL_NAME = 'cointegrated/rut5-base-absum'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/753 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/932M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/808k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/315 [00:00<?, ?B/s]

In [20]:
def summarize(
    text, n_words=None, compression=None,
    max_length=1000, num_beams=3, do_sample=False, repetition_penalty=10.0, 
    **kwargs
):
    """
    Summarize the text
    The following parameters are mutually exclusive:
    - n_words (int) is an approximate number of words to generate.
    - compression (float) is an approximate length ratio of summary and original text.
    """
    if n_words:
        text = '[{}] '.format(n_words) + text
    elif compression:
        text = '[{0:.1g}] '.format(compression) + text
    # x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    x = tokenizer(text, return_tensors='pt', padding=True)
    with torch.inference_mode():
        out = model.generate(
            **x, 
            max_length=max_length, num_beams=num_beams, 
            do_sample=do_sample, repetition_penalty=repetition_penalty, 
            **kwargs
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [21]:
summ_list = []
for top in top_texts_list:
    summ_list.append(summarize(' '.join(list(top))))

In [22]:
top_texts_list

[('пресс-конференция шансы сборной россии по хоккею на олимпиаде - 2010',
  'подготовка сборной россии по бобслею и скелетону к участию в олимпийских играх 2010 года',
  'федерация бобслея и скелетона россии поможет скворцовой в подаче иска'),
 ('страницы из чехова: хамелеон для сапожника из полтавской губернии',
  'юбилей чехова станет проверкой нашей нужды в нем - лев аннинский',
  'исаак дунаевский главный по саундтрекам в советском союзе'),
 ('груз ожиданий может придавить мюррея в финале ао-2010 - федерер',
  'экс-чемпионка мира гимнастка дос сантос дисквалифицирована за допинг',
  'у оргкомитета олимпиады-2010 нет альтернативы проблемной горе сайпресс'),
 ('кинопремию золотой орел вручат в москве',
  'миллионер из трущоб получил кинопремию золотой орел',
  'олег янковский посмертно награжден почетным золотым орлом'),
 ('сборная россии вышла в финал чемпионата мира по бенди',
  'исландцы стали бронзовыми призерами че по гандболу',
  'атлетико вышел в полуфинал кубка испании по фут

In [23]:
summ_list

['Сборная россии по хоккею и скелетону на олимпийских играх 2010 года готовится к участию в Олимпийских играх.',
 'В юбилее Чехова станет проверкой нашей нужды в нем.',
 'В финале Олимпиады-2010 Мюррея может принять участие в чемпионате мира гимнастка дос сантос, которая дисквалифицирована за допинг.',
 'Олег янковский посмертно награжден почетным золотым орлом.',
 'Исландцы стали бронзовыми призерами чемпионата мира по бенди.',
 'У берегов Вануату произошло землетрясение магнитудой 6,0.',
 'В Бахрейн будет представлена концепция развития гтк России.',
 'Суд в москве рассмотрит иск внука сталина к Эхо Москвы.',
 'Россиянка погибла при крушении эфиопского лайнера в Триполи.',
 'В день рождения чехова медведев предложил обсудить проблемы театра.',
 'Встреча по афганистану с российским президентом Владимиром Рогозиным в рамках встречи между Россией и Сша, которая состоится на заседании Совета безопасности страны.',
 'В юар плишкова поменяла бы победу на Australian Open-2010.',
 'В Воронеж